In [2]:
# coding:utf8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None) # 显示所有列
pd.set_option('display.max_rows', 10) # 设定df全部显示的行数

# 一个cell做多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

## GroupBy机制
> 1、主要是series或者是dataframe的groupby函数，针对不同的列进行分组，在分组的基础上对某一列或者某几列进行聚合运算；    
> 2、如果是对一列进行聚合运算，使用自带的聚合函数(mean,avg,min,max等)或者是agg函数（可以进行多种运算或者指定运算后的字段名称，根据参数样式而定）；    
> 3、如果是对多列进行聚合运算，使用apply函数调用自定义聚合函数或者是自带的聚合函数；    
> 4、provt_table(透视表) 和cross_tab（交叉表）分组聚合属于特殊的聚合，均可以使用groupby函数实现，只不过使用该函数更方便一些 

###   单分组 和 多分组
- 1、单分组：可以先选择数据，然后进行分组；也可以先分组然后对数据进行过滤     
- 2、多分组：形成层次化的索引，使用unstack()函数转换     
- 3、既可以使用已有的数据进行分组，也可以使用给定的数据进行分组（相当于有增一个分组列，最好与原数据有对应关系）    
- 4、分组对象均有size方法  grouded.size()
- 5、GroupBy的对象支持迭代，产生一组二元数组s

In [6]:
# 示例数据
df = pd.DataFrame({
        'key1':['a','a','b','b','a'],
        'key2':['one','two','one','two','one'],
        'data1':np.random.randn(5),
        'data2':np.random.randn(5)
    })

df.head()

# 单分组
grouped = df['data1'].groupby(df['key1'])  #grouped = df.groupby('key1')['data1']
grouped.mean()

# 多分组 
df['data1'].groupby([df['key1'], df['key2']]).mean().unstack()

# 分组键可以为列，也可以玩长度适当（多分组中能构成层次索引）的数组
states = np.array(['Ohio', 'California', 'California', 'Ohio','Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

,key1,key2,data1,data2
0,a,one,-0.817649,1.820635
1,a,two,-0.161380,-0.639129
2,b,one,0.885222,0.117331
3,b,two,-0.344257,0.967203
4,a,one,0.470980,0.587107


key1
a   -0.169350
b    0.270482
Name: data1, dtype: float64

key2,one,two
key1,,
a,-0.173334,-0.161380
b,0.885222,-0.344257


California  2005   -0.161380
            2006    0.885222
Ohio        2005   -0.580953
            2006    0.470980
Name: data1, dtype: float64

###  对分组进行迭代 
groupby对象支持迭代，会产生一个二元元祖（由分组名和数据块组成） 
- groupby 默认在axis=0(纵轴)上进行分组，也可以在其他轴上进行分组 
- 可以对分组的结果选取一个或者一组列（注意，如果是单个列，最好也以[[]]形式进行选取,这样返回的结果是df形式） df.groupby('key1')[['data1']]   
- 可以根据字典或者series数组进行分组
单独定义分组列，不过如果没有映射，感觉意义不大    
- 根据函数进行分组    
主要针对的是index上的函数，根据其返回结果进行分组（可以简化数据的处理流程）   
- 根据索引级别进行分组
针对的是层次化索引，使用参数level进行识别（level=索引名称或者级别编号 从0开始

In [11]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)
    
for (key1,key2),group in df.groupby(['key1','key2']):
    print((key1,key2))
    print(group)

# 根据函数进行分组 
df1 = df.set_index(['key2'])
def isE(str1):
    return 'e' in str1
df1.groupby(isE).sum()

a
  key1 key2     data1     data2
0    a  one -0.817649  1.820635
1    a  two -0.161380 -0.639129
4    a  one  0.470980  0.587107
b
  key1 key2     data1     data2
2    b  one  0.885222  0.117331
3    b  two -0.344257  0.967203
('a', 'one')
  key1 key2     data1     data2
0    a  one -0.817649  1.820635
4    a  one  0.470980  0.587107
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.16138 -0.639129
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.885222  0.117331
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.344257  0.967203


,data1,data2
False,-0.505638,0.328074
True,0.538553,2.525073


### 选取一列或列的子集
先聚合还是先选取的问题
> 将函数跟数组、列表、字典、Series混合使用都可以作为分分组数组  

```python
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```


In [12]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.203871
     two  -0.639129
b    one   0.117331
     two   0.967203

In [14]:
people = pd.DataFrame(np.random.randn(5, 5),
                columns=['a', 'b', 'c', 'd', 'e'],
                index=['Joe', 'Steve', 'Wes', 'Jim','Travis'])

people.iloc[2:3, [1, 2]] = np.nan 

people

# 直接使用字典进行分组
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,a,b,c,d,e
Joe,-0.618410,1.024224,0.795016,-0.174831,-1.812261
Steve,0.359038,0.072579,0.252139,-1.017353,-0.962798
Wes,1.126835,NaN,NaN,-1.036335,0.705399
Jim,0.817125,1.086428,-0.267917,-2.360611,-0.693967
Travis,-0.681819,-0.692809,1.793964,-0.748720,0.538945


,blue,red
Joe,0.620185,-1.406446
Steve,-0.765215,-0.531182
Wes,-1.036335,1.832234
Jim,-2.628528,1.209586
Travis,1.045245,-0.835683


### 根据索引级别分组
将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西在内部都会被转换为数组;    
主要是level参数的使用（层次索引级别，最外层为0，也可使用名称来指定）

In [18]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP','JP'],
                         [1, 3, 5, 1, 3]],
                        names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df.head()

hier_df.groupby(level='city', axis=1).count() # 相当于level=0

cty          US                            JP          
tenor         1         3         5         1         3
0      1.778823  0.395398  1.819212  0.864476  2.501751
1      0.991311 -0.678881  0.240655 -0.161337 -0.777650
2      1.319224 -0.536798 -0.577932 -0.255106  0.751054
3     -1.341173 -1.455722 -0.312003 -0.081164  0.053050

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合
1、主要针对的是数值型的列    
2、可以使用自带的聚合运算方法(直接调用或者使用agg/apply调用)，也可以自定义（只能agg/apply调用）    
3、对单列或者多列使用的调取参数不同（agg/apply）

quantile可以计算Series或DataFrame列的样本分位数,但其并非严格的聚合函数；类似的还有describe()函数

常见的聚合运算：  

函数名| 说明
--|--
count|分组中非NA值的数量
mean|
median| 非NA值的算术中位数
std、var| 标准差和方差
min/max|
prod| 非NA值的积
first/last| 第一个和最后一个非NA值 

In [20]:
df.head()

grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

grouped.describe()

,key1,key2,data1,data2
0,a,one,-0.817649,1.820635
1,a,two,-0.161380,-0.639129
2,b,one,0.885222,0.117331
3,b,two,-0.344257,0.967203
4,a,one,0.470980,0.587107


key1
a    0.344508
b    0.762274
Name: data1, dtype: float64

,data1,data2
key1,,
a,1.288629,2.459764
b,1.229479,0.849872


data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.169350  0.644352 -0.817649 -0.489514 -0.161380  0.154800   
b      2.0  0.270482  0.869373 -0.344257 -0.036888  0.270482  0.577852   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.470980   3.0  0.589537  1.229884 -0.639129 -0.026011  0.587107   
b     0.885222   2.0  0.542267  0.600950  0.117331  0.329799  0.542267   

                          
           75%       max  
key1                      
a     1.203871  1.820635  
b     0.754735  0.967203

### 面向列的多函数应用
使用groupby agg方法。参数为dict形式的最为方便~ 可以为每一列指定统计方法，也可每一列指定多个统计方法
```python
grouped.agg({'col_A' : np.max, 'col_B' : 'sum'}) 
grouped.agg({'col_A' : ['min', 'max', 'mean', 'std'], 'col_B' : 'sum'})
```

In [22]:
# 餐馆消费示例
tips = pd.read_csv('./examples/tips.csv')
# tips.head()
# 为防止字段名引起冲突，修改字段名
tips.rename(columns={'size':'sizes'},inplace=True)
# tips.head()

# 添加“小费占总额百分比”的列
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head()

# 进行多分组
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')  # 单一统计值
grouped_pct.agg(['mean', 'std', peak_to_peak])  # 多分组的 单列 多统计值
# 传入的是一个由(name,function)元组组成的列表，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序映射）
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) # 还是多分组 的 单列 多统计值，只不过可以指定统计后的列名


,total_bill,tip,smoker,day,time,sizes,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [25]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions) # 多列结果多种统计值  这样写 会为每一列统计多种结果值（有些是不必要的 而且结果列名无法自定义）
result
result['tip_pct']

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [27]:
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples) # 可指定结果列名 但还是无法做到为每一个统计列指定一种统计方法（dict参数可以吗，但是现在不支持了）


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

D:\Anaconda2_5.1.0_setup\envs\python36\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


KeyError: 'size'

In [35]:
grouped = tips.groupby(['day', 'smoker'])
grouped.agg({'tip_pct' : np.max, 'total_bill' : 'sum'}) # 这种方法最灵活，但是不建议使用
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'total_bill' : 'sum'})  # 一列 有多个统计值

tip_pct  total_bill
day  smoker                      
Fri  No      0.187735       73.68
     Yes     0.263480      252.20
Sat  No      0.291990      884.78
     Yes     0.325733      893.62
Sun  No      0.252672     1168.88
     Yes     0.710345      458.28
Thur No      0.266312      770.09
     Yes     0.241255      326.24

tip_pct                               total_bill
                  min       max      mean       std        sum
day  smoker                                                   
Fri  No      0.120385  0.187735  0.151650  0.028123      73.68
     Yes     0.103555  0.263480  0.174783  0.051293     252.20
Sat  No      0.056797  0.291990  0.158048  0.039767     884.78
     Yes     0.035638  0.325733  0.147906  0.061375     893.62
Sun  No      0.059447  0.252672  0.160113  0.042347    1168.88
     Yes     0.065660  0.710345  0.187250  0.154134     458.28
Thur No      0.072961  0.266312  0.160298  0.038774     770.09
     Yes     0.090014  0.241255  0.163863  0.039389     326.24

### 以“没有行索引”的形式返回聚合数据
groupby 函数参数 as_index=False:分组列不作为结果的行索引，避免层次索引的需要进一步操作；

In [36]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,sizes,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## apply:一般性的拆分-应用-合并
> apply调用函数的主参数，是每个分组对象（该点需要注意）

apply方法会将待处理的对象拆分为多个片段，然后对各个片段调用传入的参数，最后尝试将各个片段组合在一起（结果是以分组列为索引）  
如果apply方法调用的是自定义函数，且分组采用的是普通列，会形成MultiIndex  

- 可以使用group_keys=False禁止使用分组键做索引， 避免产生层次化索引 

**疑惑：**
<span class="burk">如果调用的外部函数如下面的top函数，在apply的过程中需要外部传参该怎么办？</span>

In [38]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

tips.groupby('smoker').apply(top) # 对groupby的每一个子对象适用于指定的函数， 然后将结果组装在一起

,total_bill,tip,smoker,day,time,sizes,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


total_bill   tip smoker   day    time  sizes   tip_pct
smoker                                                            
No     88        24.71  5.85     No  Thur   Lunch      2  0.236746
       185       20.69  5.00     No   Sun  Dinner      5  0.241663
       51        10.29  2.60     No   Sun  Dinner      2  0.252672
       149        7.51  2.00     No  Thur   Lunch      2  0.266312
       232       11.61  3.39     No   Sat  Dinner      2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner      2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner      4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner      1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner      2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner      2  0.710345

### 禁止分组键
分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将group_keys=False传入groupby即可禁止该效果,功能有些类似as_index

In [39]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,sizes,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析 
> 所谓的分位数和桶分析就是将数据按照分位数(quantile)或者桶(bucket)进行拆分，然后groupby分组   
> 主要会用到df的一些特殊工具方法：cut()等长分组切分函数   qcut()分位数切分函数  

也可以将数据按照指定的规则进行类别划分，然后根据类别进行分组

- pd.cut 将数据集切分为几组（几类） 类似于hist的bins；其对象可以直接用于groupby 

In [44]:
frame = pd.DataFrame({
        'data1': np.random.randn(100),
        'data2': np.random.randn(100)
    })
factor = pd.cut(frame.data1,4)

factor[:10]

def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

0     (-1.0, 0.258]
1     (-1.0, 0.258]
2    (0.258, 1.517]
3    (0.258, 1.517]
4    (0.258, 1.517]
5    (0.258, 1.517]
6     (-1.0, 0.258]
7     (-1.0, 0.258]
8     (-1.0, 0.258]
9     (-1.0, 0.258]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.263, -1.0] < (-1.0, 0.258] < (0.258, 1.517] < (1.517, 2.775]]

,count,max,mean,min
data1,,,,
"(-2.263, -1.0]",14.0,0.856908,-0.235680,-1.635152
"(-1.0, 0.258]",46.0,1.666967,-0.144909,-2.884900
"(0.258, 1.517]",35.0,1.417801,-0.157629,-1.658244
"(1.517, 2.775]",5.0,1.555605,0.134348,-0.528442


### 示例：用特定与分组的值填充缺失值 
> eg:用每组剩余值的均值 或者 指定每组缺失值的填充值 来填充改组的缺失值（比直接使用整体均值填充缺失值效果要好一些）  

In [45]:
states = list('ABCDEFGH')
group_keys = ['East']*4 + ['West']*4
data = pd.Series(np.random.randn(8), index=states)
data[['B','D','G']] = np.nan
data.groupby(group_keys).mean()  # 虽然group_keys不在被分组的数据中，二者默认是根据index进行关联后然后在分组

# 定义分组填充函数
fill_mean = lambda g:g.fillna(g.mean())
data.groupby(group_keys).apply(fill_mean)

# 也可在不同分组中指定填充值
fill_values = {'East':0.5,'West':-1}  # 在不同的组别指定要替换NA的值
fill_func = lambda g:g.fillna(fill_values[g.name])  # 每个分组对象有name、size()等系统属性，可以善加利用
data.groupby(group_keys).apply(fill_func)

East    1.070455
West   -0.094599
dtype: float64

A    1.077760
B    1.070455
C    1.063151
D    1.070455
E    0.973893
F   -0.397449
G   -0.094599
H   -0.860240
dtype: float64

A    1.077760
B    0.500000
C    1.063151
D    0.500000
E    0.973893
F   -0.397449
G   -1.000000
H   -0.860240
dtype: float64

### 示例：随机采样和排列 
> 这一块涉及到样本的选择和模型的交叉验证，需要注意        

- 方法:使用np.random.permutation(N)[k:] N是完整数据的大小，k是期望样本的大小  

In [48]:
# 构建基础扑克牌数据  以牌名（大小+花色）为索引；以数值（0-10，JQK=10）为数据
# 红桃 Hearts 黑桃 Spades 梅花 Clubs  方片 Diamonds
suits = ['H','S','C','D']
card_val = (list(range(1, 11)) + [10] * 3) * 4 # list 相加即为list.append(list)
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)

deck = pd.Series(card_val, index=cards)    
deck[:13]

# 定义随机抽取函数  随机在draw数据中抽取n个值（放回的，没哟其他的限制条件）
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck)))[:n]

draw(deck)

# 定义从每种花色随机抽取两张 
get_suit = lambda card: card[-1] # 获取花色
deck.groupby(get_suit).apply(draw, n=2)  # 对series而言，groupby针对的是其index进行分组

# 另一种方法
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

AH      1
2H      2
3H      3
4H      4
5H      5
       ..
9H      9
10H    10
JH     10
KH     10
QH     10
Length: 13, dtype: int64

5S    5
8C    8
9C    9
2H    2
6C    6
dtype: int64

C  5C      5
   10C    10
D  10D    10
   AD      1
H  10H    10
   JH     10
S  2S      2
   10S    10
dtype: int64

2C     2
3C     3
QD    10
KD    10
6H     6
7H     7
7S     7
8S     8
dtype: int64

### 示例：分组加权平均数和相关系数 
> 计算df列与列之间、两个series之间的分组加权、相关系数等标准化作业 

In [49]:
df = pd.DataFrame({
        'category':['a']*4+['b']*4,
        'data': np.random.randn(8),
        'weights':np.random.rand(8)
    })
grouped = df.groupby('category')
# 计算分组加权平均数
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.024755
b    0.722270
dtype: float64

In [50]:
close_px = pd.read_csv('./examples/stock_px.csv',parse_dates=True, index_col=0)
close_px.head()

# 计算一个由日收益率（通过百分数变化计算）与SPX之间的年度相关系数组成的df
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x:x.year)
by_year.apply(spx_corr).head()

# 计算相关系数
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT'])).head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25
1990-02-06,5.01,8.12,2.88,17.56,4.32,0.51,6.15,329.66,6.23
1990-02-07,5.04,7.77,2.91,17.93,4.38,0.51,6.17,333.75,6.33


,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078


1990    0.408271
1991    0.266807
1992    0.450592
1993    0.236917
1994    0.361638
dtype: float64

### 示例：组级别的线性回归

In [51]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617
...,...,...
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261


## 透视表与交叉表
- 透视表：pivot_table，选择两个系列(单列或者多个列构成的list)形成横纵坐标，对某一列或者某几列进行计算、累加、求和等运算   参数有data, index, columns, aggfunc等 pd和df均有该函数，不过一般都是df调用    
```python
pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
```
- 交叉表：crosstab  用于计算分组频数（每个分组项的个数，使用pivot_table+aggfunc=len也能实现）的特殊透视表，pd级别函数   
```python  
# 该函数的values参数基本上是不用的
crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, dropna=True, normalize=False)
```

> 这两个函数是一种更特殊意义上的分组函数；均可使用groupby函数实现；这两个函数使用场景较多但是使用有较大限制，所以单独出来，方便使用

In [52]:
tips.pivot_table(index=['day', 'smoker'])

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker')

# fill_value参数对缺失值进行填充
tips.pivot_table(values=['tip_pct','sizes'], index=['time','day'], columns='smoker',aggfunc=sum,margins=True)  # all值根据aggfunc保持一致
tips.pivot_table('tip_pct', index=['time', 'smoker'],columns='day', aggfunc=len, margins=True)

# 对分组进行个数交叉统计
pd.crosstab([tips.time,tips.day], tips.smoker, margins=True)

sizes                tip_pct                      
smoker          No    Yes  All         No        Yes        All
time   day                                                     
Dinner Fri     6.0   20.0   26   0.418867   1.488126   1.906993
       Sat   115.0  104.0  219   7.112145   6.212055  13.324199
       Sun   167.0   49.0  216   9.126438   3.557756  12.684194
       Thur    2.0    NaN    2   0.159744        NaN   0.159744
Lunch  Fri     3.0   11.0   14   0.187735   1.133620   1.321354
       Thur  110.0   40.0  150   7.053669   2.785676   9.839345
All          403.0  224.0  627  24.058598  15.177232  39.235830

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244